In [4]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
import os

In [5]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# === CONFIGURACIÓN ===
CARPETA_MARKDOWN = "TSpec-LLM/3GPP-clean/Rel-15"  # Cambia esta ruta si es necesario
MODELO_EMBEDDINGS = "sentence-transformers/all-MiniLM-L6-v2"
INDEX_FAISS = "faiss_tspec"
USAR_INDEX_EXISTENTE = True  # Cambia a False si quieres recrear el índice
from dotenv import load_dotenv
import sys

load_dotenv()  # Carga automáticamente el archivo .env

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("❌ ERROR: No se encontró OPENAI_API_KEY en el archivo .env.")
    sys.exit(1)

In [6]:
def cargar_documentos():
    print("📥 Cargando documentos Markdown...")
    loader = DirectoryLoader(CARPETA_MARKDOWN, glob="**/*.md", loader_cls=TextLoader, show_progress=True)
    documentos = loader.load()
    print(f"✅ Se cargaron {len(documentos)} documentos.")
    return documentos

# === PASO 2: Dividir documentos ===
def dividir_en_chunks(documentos):
    print("✂️ Dividiendo en fragmentos...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documentos)
    print(f"✅ Se generaron {len(chunks)} fragmentos.")
    return chunks

# === PASO 3: Crear o cargar FAISS ===
def construir_faiss(chunks):
    print("🧠 Generando embeddings en batches y construyendo FAISS...")

    embeddings = HuggingFaceEmbeddings(
        model_name=MODELO_EMBEDDINGS,
        encode_kwargs={
            "batch_size": 64,
        }
    )

    db = FAISS.from_documents(chunks, embeddings)
    db.save_local(INDEX_FAISS)

    print("✅ FAISS guardado.")
    return db

def cargar_faiss():
    print("📂 Cargando FAISS existente...")
    embeddings = HuggingFaceEmbeddings(model_name=MODELO_EMBEDDINGS)
    db = FAISS.load_local(INDEX_FAISS, embeddings, allow_dangerous_deserialization=True)
    return db

# === PASO 4: Crear agente y responder ===
def crear_agente(db):
    print("🤖 Inicializando agente...")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    retriever = db.as_retriever()
    chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return chain

def hacer_pregunta(chain):
    print("\n💬 Agente listo. Escribe tu pregunta técnica (o 'salir'):")
    while True:
        query = input("🧠> ").strip()
        if query.lower() == "salir":
            break
        respuesta = chain.invoke({"query":query})["result"]
        print(f"\n📘 Respuesta:\n{respuesta}\n")

In [7]:
documentos = cargar_documentos()

📥 Cargando documentos Markdown...


100%|█████████████████████████████████████| 1596/1596 [00:00<00:00, 1916.01it/s]

✅ Se cargaron 1596 documentos.


In [8]:
chunks = dividir_en_chunks(documentos)

✂️ Dividiendo en fragmentos...
✅ Se generaron 724953 fragmentos.


In [11]:
print(chunks[200:300])

[Document(metadata={'source': 'TSpec-LLM/3GPP-clean/Rel-15/43_series/43059-f20.md'}, page_content='Case 3: During intra-BSC HO or other intra-BSC RR management procedure,\nif a HO complete or RR management procedure completion was not received\nin BSC and the corresponding timer expired. In this case a reset or\nabort message will be sent to SMLC indicating MS timeout. The location\nservice may then restart from either the SMLC if a reset was sent or\nfrom the LCS Client, VMSC or SGSN if an abort was sent.\n\nCase 4: If an inter-BSC handover is needed during a location procedure\nor if the BSC times out on an RRLP response from the target MS, the BSC\nshall send a BSSLAP Abort to the SMLC. The location service attempt may\nthen be restarted from the LCS Client, VMSC, or SGSN.\n\n### 9.4.5 Error Handling for the SMLC in CS Domain\n\n![](media/image17.wmf){width="6.676388888888889in"\nheight="4.832638888888889in"}\n\nFigure 32: Error Handling for the SMLC in the CS Domain\n\n### 9.4.5a E